In [3]:
import numbers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

%config Completer.use_jedi = False

def get_mape(y_predict, y_true):
    return (abs(y_predict - y_true) / y_true).mean()

def process_rooms_number(x):
        
    if pd.isna(x):
        return 1
    
    if isinstance(x, int):
        return x
    
    if x.isdigit():
        return int(x)
    
    if x == 'Студия':
        return 1
    
    if x == 'Своб. планировка':
        return 1
    
    if x == '> 9':
        return 10

    return 1

<h3>Реализуем линейную регрессию</h3>

<p>Чаще всего алгоритмы машинного обучения реализуются в виде классов с обязательными методами <code>.fit()</code>, <code>.predict()</code>. </p>

<p><code>.fit()</code> – обучить алгоритм на обучающей выборке;</p>

<p><code>.predict()</code> – сделать предсказание на тестовых данных.</p>

<p> </p>

In [20]:
class LinearRegression:
    
    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001, print_every=100, batch_size=8192):
        
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        self.batch_size = batch_size
        
        self.weights = None
        self.bias = None
        
        
    def fit(self, X_train, y_train, X_val, y_val):
        
        self.check_regression_X_y(X_train, y_train)
        self.check_regression_X_y(X_val, y_val)
        
        n, m = X_train.shape
        
        self.weights = np.ones((m, 1))
        self.bias = y_train.mean()
        
        n_iter = 0
        gradient_norm = np.inf
        
        while n_iter < self.max_iter and gradient_norm > self.tol:

            indecies = np.random.randint(low=0, high=X_train.shape[0], size=self.batch_size)
            dJdw, dJdb = self.grads(X_train[indecies], y_train[indecies])
                
            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))
                
            self.weights = self.weights - self.lr * dJdw
            self.bias = self.bias - self.lr * dJdb
            
            n_iter += 1
            
            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)
        
        return self

    def predict(self, X):
        return X @ self.weights + self.bias
    
    def grads(self, X, y):

        y_hat = self.predict(X)
        
        dJdw = ((y_hat - y) * X).mean(axis=0, keepdims=True).T
        dJdb = (y_hat - y).mean()

        self.check_grads(dJdw, dJdb)
        
        return dJdw, dJdb
    
    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):
        
        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)
        
        MAPE_train = get_mape(train_preds, y_train)
        MAPE_val = get_mape(val_preds, y_val)
        
        print(f'{n_iter} completed. MAPE on train: {MAPE_train}, val: {MAPE_val},  grad norm: {gradient_norm}')
        
        
    def check_grads(self, dJdw, dJdb):
        
        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным '
                             f'числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')
            
        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')
            
        
    @staticmethod
    def check_regression_X_y(X, y):
        
        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')
            
        if np.isnan(X).any():
            raise ValueError(f'X не должен содержать "not a number" (np.nan)')
            
        if np.isnan(y).any():
            raise ValueError(f'y не должен содержать "not a number" (np.nan)')
        
        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')
            
        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')
                    
        if np.any([(not isinstance(value, numbers.Real)) for value in y.flatten()]):
            raise ValueError(f'Ответы на объектах должны быть действительными числами!')
       

<h3>Тестируем модель на простой задаче</h3>

In [7]:
X = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 1],
])
y = np.array([[1], [2], [3], [4]])
model = LinearRegression(lr=0.1)
model.fit(X, y, X, y)
model.predict(X)

100 completed. MAPE on train: 0.05986845513923276, val: 0.05986845513923276,  grad norm: 0.04721174913987021
200 completed. MAPE on train: 0.013152958287128286, val: 0.013152958287128286,  grad norm: 0.00997014189150913
300 completed. MAPE on train: 0.0032241396452775497, val: 0.0032241396452775497,  grad norm: 0.002524994369805291


array([[1.00227301],
       [2.00218537],
       [3.00209773],
       [3.99621483]])

<h3>Решаем задачу предсказания цены</h3>

In [8]:
data = pd.read_csv('real_estate_novosibirsk.csv')

<p>Чистим данные:</p>

In [9]:
data = data.drop_duplicates(subset=['item_id'], keep='last')
data = data.dropna(subset=['area'])
data['rooms_number'] = data['rooms_number'].apply(process_rooms_number).copy()
data = data[(data.price > 970000) & (data.price < 12700000)]
data = data[(data.floor < 59)]

data = data.dropna(axis=0)

In [10]:
train, val, train_price, val_price = train_test_split(data.drop('price', axis=1), data['price'], random_state=42)

In [12]:
test_data = pd.read_csv('test_price_estimator (2).csv')
test, test_price = test_data.drop('price', axis=1), test_data['price']

y_train = train_price.values.reshape(-1, 1)
y_val = val_price.values.reshape(-1, 1)

### Делаем бейзлайн

In [13]:
get_mape(y_predict=np.median(y_train), y_true=y_val)

0.37614684131639187

### Обучаем модель

1) Начинаем с простого

In [14]:
X_train = train[['area']].values
X_val = val[['area']].values


model = LinearRegression(lr=6e-4, max_iter=140000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

10000 completed. MAPE on train: 0.32495804010076984, val: 0.31584565698364414,  grad norm: 176398.15897057083
20000 completed. MAPE on train: 0.26970850832417564, val: 0.261715014421197,  grad norm: 58952.70546230126
30000 completed. MAPE on train: 0.25501556254030644, val: 0.24728627123292868,  grad norm: 19702.14146001821
40000 completed. MAPE on train: 0.2506691397472152, val: 0.24303110373999803,  grad norm: 6584.504902133548
50000 completed. MAPE on train: 0.24928668407411153, val: 0.24167296738831417,  grad norm: 2200.5579898108713
60000 completed. MAPE on train: 0.24883172794678735, val: 0.24122697532098183,  grad norm: 735.4319783332422
70000 completed. MAPE on train: 0.24868115238629024, val: 0.24107995331682502,  grad norm: 245.78320465057666
80000 completed. MAPE on train: 0.2486309578409756, val: 0.2410310124254577,  grad norm: 82.14136108844785
90000 completed. MAPE on train: 0.24861418805052538, val: 0.241014662190146,  grad norm: 27.451848107767393
100000 completed. MAPE

<p>Для того, чтобы начать ориентироваться в метрике решения задачи, очень важно построить одну или несколько простых моделей. Часто есть соблазн добавить все признаки сразу и обучить модель — мы так поступать не будем. Наоборот, мы будем постепенно добавлять признаки и следить за тем, что модель решает задачу лучше и лучше. </p>

2) Увеличиваем количество признаков

In [15]:
X_train = train[['area', 'floors_in_house', 'floor']].values
X_val = val[['area', 'floors_in_house', 'floor']].values

model = LinearRegression(lr=6e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

10000 completed. MAPE on train: 0.34071672398641273, val: 0.33139777932922787,  grad norm: 202475.9417288229
20000 completed. MAPE on train: 0.27783005179824266, val: 0.26892244418420735,  grad norm: 102561.201702077
30000 completed. MAPE on train: 0.25360754966154325, val: 0.24497668881758333,  grad norm: 51950.863913807516
40000 completed. MAPE on train: 0.24429354503583503, val: 0.2357741283603333,  grad norm: 26314.943824768892
50000 completed. MAPE on train: 0.24051512736404806, val: 0.23220770453089087,  grad norm: 13329.446641150054
60000 completed. MAPE on train: 0.23885963089488235, val: 0.2306786476480523,  grad norm: 6751.834582752558
70000 completed. MAPE on train: 0.23809728785823325, val: 0.22998164981486588,  grad norm: 3420.0422163151065
80000 completed. MAPE on train: 0.2377307046635948, val: 0.22965168927620386,  grad norm: 1732.3719380295931
90000 completed. MAPE on train: 0.23754925343209923, val: 0.2294889857950511,  grad norm: 877.5074522049489
100000 completed. M

Делаем новые признаки
##### One hot encoding

In [16]:
ohe_example = pd.DataFrame({'feature': ['a', 'b', 'a', 'c']})
ohe_example

,feature
0,a
1,b
2,a
3,c


In [17]:
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(ohe_example)

C:\Users\aleks\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [18]:
ohe_house_type_transformer = OneHotEncoder(sparse=False)
train_ohe_house_type = ohe_house_type_transformer.fit_transform(train[['type_of_house']])
val_ohe_house_type = ohe_house_type_transformer.transform(val[['type_of_house']])

ohe_district_transformer = OneHotEncoder(sparse=False)
train_ohe_district = ohe_district_transformer.fit_transform(train[['district']])
val_ohe_district = ohe_district_transformer.transform(val[['district']])

X_train_extended = np.hstack([X_train, train_ohe_house_type, train_ohe_district])
X_val_extended = np.hstack([X_val, val_ohe_house_type, val_ohe_district])

model = LinearRegression(lr=6e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train_extended, y_train, X_val_extended, y_val)

C:\Users\aleks\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\aleks\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


10000 completed. MAPE on train: 0.25853392474642173, val: 0.24946162482379938,  grad norm: 161356.2664430575


KeyboardInterrupt: 

In [21]:
model = LinearRegression(lr=6e-4, max_iter=120000, print_every=10000, tol=0.1, batch_size=8196)
model.fit(X_train_extended, y_train, X_val_extended, y_val)

10000 completed. MAPE on train: 0.34913096553477896, val: 0.33780964029429317,  grad norm: 35259039.05542406
20000 completed. MAPE on train: 0.2135001930159604, val: 0.20619694615062906,  grad norm: 11239821.60750618
30000 completed. MAPE on train: 0.2027799403839704, val: 0.19601508891327654,  grad norm: 11153157.372839529
40000 completed. MAPE on train: 0.20137631969971675, val: 0.19403988577853978,  grad norm: 9404761.145880844
50000 completed. MAPE on train: 0.22040161236665193, val: 0.21284251824393985,  grad norm: 9840230.077290695
60000 completed. MAPE on train: 0.1929194530988486, val: 0.1857037856658644,  grad norm: 13820324.576880004
70000 completed. MAPE on train: 0.36227675194042813, val: 0.3518530409200515,  grad norm: 53593580.198034845
80000 completed. MAPE on train: 0.21698369669774392, val: 0.21123091262016894,  grad norm: 40097192.51639471
90000 completed. MAPE on train: 0.20478118908796075, val: 0.19745486030538942,  grad norm: 11885837.567868296
100000 completed. MA

### Задание на семинаре: попробовать улучшить метрику MAPE до 15.8% (топ-1 без ML с первой недели).

Варианты путей для улучшения:

    1) Делать новые признаки из существующих;
    2) Препроцессинг данных, целевой переменной - постпроцессинг ответов модели;
    3) Анализ ошибок модели –> генерация идей;
    4) Добавить регуляризацию;


<h3>Задание на семинаре: реализуем логистическую регрессию</h3>

<p>Мы получаем оптимальные веса алгоритма градиентным спуском:</p>

<p style="text-align:center"><br />
<br />
<span class="math-tex">\(\begin{bmatrix} w_{1}^{t+1}\\  ...\\ w_{m}^{t+1}\\  \end{bmatrix} = \begin{bmatrix} w_{1}^{t}\\  ...\\ w_{m}^{t}\\  \end{bmatrix} - \alpha \cdot  \begin{bmatrix} \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{1}^{(i)}\\  ...\\ \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{m}^{(i)}\\  \end{bmatrix}\)</span></p>

<p style="text-align:center"><span class="math-tex">\(b^{t+1} = b^{t} - \alpha \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})\)</span></p>

<p style="text-align:center">&nbsp;</p>

<p>&nbsp;</p>

In [ ]:
from sklearn.metrics import accuracy_score


class LogisticRegression:
    
    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001, print_every=100):
        
        '''
        max_iter – максимальное количеств
        '''
        
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        
        self.weights = None
        self.bias = None
        
    def fit(self, X_train, y_train, X_val, y_val):
        
        '''
        Обучение модели.
        
        X_train – матрица объектов для обучения
        y_train – ответы на объектах для обучения
        
        X_val – матрица объектов для валидации
        y_val – ответы на объектах для валидации
        '''
        
        self.check_binary_clf_X_y(X_train, y_train)
        self.check_binary_clf_X_y(X_val, y_val)
                
        n, m = X_train.shape
        
        self.weights = 
        self.bias = 
        
        n_iter = 0
        gradient_norm = np.inf
        
        while n_iter < self.max_iter and gradient_norm > self.tol:
            
            dJdw, dJdb = self.grads(X_train, y_train)
            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))
                
            self.weights = 
            self.bias = 
            
            n_iter += 1
            
            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)
        
        return self
    
    def predict(self, X):  
        
        '''
        Метод возвращает предсказанную метку класса на объектах X
        '''
        
        pass
        
    
    def predict_proba(self, X):
        
        '''
        Метод возвращает вероятность класса 1 на объектах X
        '''
        pass
    
    def grads(self, x, y):
        
        '''
        Рассчёт градиентов
        '''
        y_hat = 
        
        dJdw = 
        dJdb = 
        
        self.check_grads(dJdw, dJdb)
        
        return dJdw, dJdb
    
    @staticmethod
    def sigmoid(x):
        '''
        Сигмоида от x
        '''
        pass
    
    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):
        
        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)
        
        train_acc = accuracy_score(train_preds, y_train)
        val_acc = accuracy_score(val_preds, y_val)
        
        print(f'{n_iter} completed. accuracy_score on train: {train_acc}, val: {val_acc}, grad_norm: {gradient_norm}')
        
    def check_grads(self, dJdw, dJdb):
        
        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным'
                             f' числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')
            
        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')
    
    @staticmethod
    def check_binary_clf_X_y(X, y):
        
        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')
            
        if np.isnan(X).any():
            raise ValueError(f'X не должен содержать "not a number" (np.nan)')
            
        if np.isnan(y).any():
            raise ValueError(f'y не должен содержать "not a number" (np.nan)')
        
        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')
            
        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')

                    
        if sorted(np.unique(y)) != [0, 1]:
            raise ValueError(f'Ответы на объектах должны быть только 0 или 1, а у нас np.unique(y) = {np.unique(y)}')


<h2>Домашнее задание</h2>

<p>Воспользуемся реализованной моделью логистической регрессии, чтобы решить задачу определения пола пользователя Авито.</p>

<p><a href="https://stepik.org/media/attachments/lesson/527992/binary_clf_data.csv" rel="noopener noreferrer nofollow">Данные</a> даны в сыром виде &ndash; айтемы и их категории, которые выкладывали покупатели на Авито. Целевая переменная: <em>gender.</em></p>

<p>Вам необходимо разбить данные на train, val. Перед загрузкой файла с ответом убедитесь, что точность (<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html" rel="noopener noreferrer nofollow">accuracy</a>)&nbsp;на валидации не менее 0.7.</p>

<p>&nbsp;</p>

<p><strong>План действий</strong></p>

<p>Сначала нужно преобразовать категории с помощью one-hot encoding. Далее необходимо агрегировать категории, в которых пользователи выкладывали объявления, чтобы получить вектор признаков для каждого объекта. В результате у каждого пользователя будет вектор признаков, содержащий количество айтемов, выложенных в каждой из возможных категорий.</p>

<ul>
	<li>Убедитесь, что для каждого пользователя в выборке есть только один объект, каждый признак означает количество айтемов, выложенное этим пользователем в категории;</li>
	<li>Убедитесь, что после one-hot энкодинга каждая категория соответствует признаку,&nbsp;<strong>одинаковому в train, val и test.</strong></li>
</ul>

<p>Попробуйте варианты отбора признаков. Для борьбы с переобучением на редких категориях используйте регуляризацию. В качестве&nbsp;ответа загрузите файл с предсказанием пола для пользователей:</p>

<p style="text-align:center">&nbsp;</p>

<table align="center" border="1" cellpadding="1" cellspacing="1" style="width:500px">
	<thead>
		<tr>
			<th style="text-align:center">user_id</th>
			<th style="text-align:center">gender</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td style="text-align:center">15424171</td>
			<td style="text-align:center">male</td>
		</tr>
		<tr>
			<td style="text-align:center">15454025</td>
			<td style="text-align:center">female</td>
		</tr>
	</tbody>
</table>

<p style="text-align:center">&nbsp;</p>

<p>Такой файл можно сформировать с помощью&nbsp;<code>test_predictions.to_csv(&#39;test_predictions.csv&#39;, index=False)</code>.</p>

<p>После того, как получилось обучить модель, ответьте на вопрос: какие из категорий вносят наибольший вклад в вероятность класса &quot;мужчина&quot; и класса &quot;женщина&quot;?</p>

<p>Например, если вы закодировали &quot;мужчина&quot; как 1, большие положительные веса при признаках будут означать большой вклад в вероятность класса 1, большие по модулю отрицательные веса будут вносить наибольший вклад в вероятность класса 0. Согласуется ли полученный результат с вашим жизненным опытом?</p>
